In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterstats as rs

Convert to intergers

In [ ]:
# Define the input and output directories
input_folder = r".\LUH2\Clipped"
output_folder = r".\LUH2\Int_Folder"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all raster files in the input folder
raster_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Loop through each raster file and convert to integer type
for raster_file in raster_files:
    input_raster = os.path.join(input_folder, raster_file)
    output_raster = os.path.join(output_folder, raster_file)

    # Open the raster file
    with rasterio.open(input_raster) as src:
        # Read the data
        data = src.read(1)  # Read the first band

        # Get the NoData value
        nodata_value = src.nodata

        # Handle NoData values
        if nodata_value is not None:
            data = np.where(data == nodata_value, -9999, data)

        int_data = data.astype(np.int8)  # Change np.int32 to np.int64 if needed

        # Define metadata for the output raster
        metadata = src.meta.copy()
        metadata.update({
            'dtype': 'int32',  # Change to 'int64' if needed
            'count': 1,
            'nodata': -15  # Set the NoData value for the output raster
        })

        # Write the converted data to a new raster file
        with rasterio.open(output_raster, 'w', **metadata) as dst:
            dst.write(int_data, 1)

    print(f"Converted {input_raster} to {output_raster}")

print("All raster files have been converted and plotted.")

Calculate Zonal Statistics

In [ ]:
# Define the input and output directories
input_folder = r".\LUH2\Int_Folder"
point_feature_class = r".\CLM_Testing.gdb\Point_LUH2_XyToPoint.shp"
output_folder = r".\LUH2\Zonal_Stats"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the point feature class as a GeoDataFrame
gdf = gpd.read_file(point_feature_class)

# Ensure the dataset has a unique ID column
point_id_field = "pointid"  # Modify this if the ID field is different

# List all TIFF files in the input folder
tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Loop through each TIFF file and calculate zonal statistics
for tiff_file in tiff_files:
    input_tiff = os.path.join(input_folder, tiff_file)
    
    # Compute zonal statistics
    zonal_stats = rs.zonal_stats(
        gdf, input_tiff, stats=["max"], geojson_out=True
    )

    # Convert results to a DataFrame
    df = pd.DataFrame([
        {
            point_id_field: feature["properties"][point_id_field],
            "MaxValue": feature["properties"]["max"]
        }
        for feature in zonal_stats
    ])

    # Define the output file path (CSV instead of DBF)
    output_csv = os.path.join(output_folder, f"ZonalStats_{os.path.splitext(tiff_file)[0]}.csv")
    df.to_csv(output_csv, index=False)

    print(f"Zonal statistics calculated for {input_tiff} and saved to {output_csv}")

print("All zonal statistics have been calculated.")


Merge files

In [ ]:
# Define the directory containing the DBF files
output_folder = r".\LUH2\Zonal_Stats"
merged_output_folder = r".\LUH2\MSE_calculation"

# Create necessary output directories
os.makedirs(output_folder, exist_ok=True)
os.makedirs(merged_output_folder, exist_ok=True)


# Initialize an empty list to hold DataFrames
dataframes = []

# Loop through all CSV files in the output directory
for filename in os.listdir(output_folder):
    if filename.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(os.path.join(output_folder, filename))
        
        # Get the base name of the file (without extension)
        base_name = os.path.splitext(filename)[0]
        
        # Rename columns that contain 'MaxValue' to include the base name
        df.columns = df.columns.str.replace('MaxValue', f'{base_name}', case=False)
        
        # Remove the specific substring from column names
        df.columns = df.columns.str.replace('ZonalStats_Clipped_Multiplied_states_', '', case=False)
        
        # Drop columns that contain 'COUNT' or 'Area'
        df = df.loc[:, ~df.columns.str.contains('COUNT|Area', case=False)]
        
        # Check for duplicate columns and keep the first occurrence
        df = df.loc[:, ~df.columns.duplicated(keep='first')]

        dataframes.append(df)

# Merge all DataFrames on 'pointid'
if dataframes:
    merged_df = dataframes[0]
    for df in dataframes[1:]:
        merged_df = pd.merge(merged_df, df, on='pointid', how='outer', suffixes=('', '_dup'))
        
        # Remove duplicate columns that may have been created during the merge
        merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]

    # Save the merged DataFrame to a new CSV file in the merged output directory
    output_file_path = os.path.join(merged_output_folder, 'merged_output.csv')
    merged_df.to_csv(output_file_path, index=False)

    print(f"Merging and cleaning completed. Output saved as '{output_file_path}'.")
else:
    print("No CSV files found to merge.")

In [ ]:
# Define file paths
workspace = r"./"
gdb_path = os.path.join(workspace, "CLM_Testing.gdb")
input_raster = os.path.join(gdb_path, "CellSta_MSD_2008.tif")  # Assuming the raster is extracted as a .tif
vector_layer = os.path.join(gdb_path, "Point_LUH2_ReProjectRaster.shp")  # Assuming it's a Shapefile
output_csv = os.path.join(workspace, "Tabulate_LUH_MSD_Classes.csv")

# Step 1: Get the cell size of the raster
try:
    with rasterio.open(input_raster) as raster:
        cell_size = raster.res[0]  # Assuming square pixels
        print(f"Processing cell size set to: {cell_size}")
except Exception as e:
    print(f"Error reading raster file: {str(e)}")
    exit()

# Step 2: Perform Tabulate Area (Zonal Statistics)
try:
    # Load the vector layer (geospatial points or polygons)
    gdf = gpd.read_file(vector_layer)

    # Compute zonal statistics (counts number of pixels per category)
    stats = zonal_stats(gdf, input_raster, stats=["count"], categorical=True, geojson_out=True)

    # Convert results to a DataFrame
    tabulate_df = pd.DataFrame([
        {**{"ID": feature["properties"]["ID"]}, **feature["properties"]}
        for feature in stats
    ])

    # Save the output table as CSV
    tabulate_df.to_csv(output_csv, index=False)
    print(f"Tabulate Area completed successfully. Output saved to {output_csv}")

except Exception as e:
    print(f"Error during zonal statistics calculation: {str(e)}")


Processing cell size set to: 30.0000016640515
Tabulate Area completed successfully.


In [ ]:
# File paths
output_folder = r".\LUH2\MSE calculation"
exported_table = os.path.join(output_folder, "Tabulate_LUH_MSD_Classes.csv.csv")
existing_table = os.path.join(output_folder, "merged_output.csv")
merged_output = os.path.join(output_folder, "final_merged_output_V_01_25.csv")

# Check if both tables exist
if os.path.exists(exported_table) and os.path.exists(existing_table):
    # Load the tables as pandas DataFrames
    df_exported = pd.read_csv(exported_table)
    df_existing = pd.read_csv(existing_table)

    # Merge the tables on 'pointid' (left) and 'Value' (right)
    df_merged = pd.merge(df_existing, df_exported, left_on='pointid', right_on='VALUE', how='outer')

    # Save the merged table to a new CSV file
    df_merged.to_csv(merged_output, index=False)
    print(f"Tables merged successfully into {merged_output}")
else:
    print(f"One or both tables not found. Ensure both '{exported_table}' and '{existing_table}' exist.")

In [ ]:
# File paths
output_folder = r".\LUH2\MSE calculation"
merged_file = os.path.join(output_folder, "final_merged_output_V_01_25.csv")
classification_file = os.path.join(output_folder, "Raw Classification.csv")

# Check if the files exist
if os.path.exists(merged_file) and os.path.exists(classification_file):
    # Load the merged file and classification file into DataFrames
    df_merged = pd.read_csv(merged_file)
    df_classification = pd.read_csv(classification_file)
    
    # Extract the 'MSD Value' and 'Layer Name' columns from classification.csv
    # The 'MSD Value' will be used to map to 'VALUE_' columns in the merged file
    value_to_layer = {}
    
    # Iterate through rows in classification.csv to create the mapping from MSD Value to Layer Name
    for idx, row in df_classification.iterrows():
        msd_value = row['MSD Val']
        layer_name = row['Layer Name']
        value_to_layer[msd_value] = layer_name
    
    # Identify the 'VALUE_' columns in the merged DataFrame
    value_columns = [col for col in df_merged.columns if 'VALUE_' in col]
    
    # Create a dictionary to map the 'VALUE_' columns to the corresponding Layer Name
    value_column_mapping = {}
    
    for value_column in value_columns:
        # Extract the number from the 'VALUE_' column (e.g., 'VALUE_1' -> 1)
        value_number = int(value_column.split('_')[1])
        
        # Map the MSD Value to the corresponding Layer Name from classification.csv
        if value_number in value_to_layer:
            value_column_mapping[value_column] = value_to_layer[value_number]
    
    # Rename the columns in df_merged using the mapping
    df_merged.rename(columns=value_column_mapping, inplace=True)

    # Save the updated DataFrame back to CSV
    df_merged.to_csv(merged_file, index=False)
    print(f"Column names updated successfully in {merged_file}")
else:
    print(f"One or both files '{merged_file}' and '{classification_file}' do not exist.")

Convert csv file to excel, and calculate proportions

In [ ]:


# Define file paths
csv_file = r".\LUH2\MSE calculation\final_merged_output.csv"
excel_file = r".\LUH2\MSE calculation\output.xlsx"

# Step 1: Convert CSV to Excel
if os.path.exists(csv_file):
    # Load the CSV into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Save it as an Excel file
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='final_merged_output', index=False)
    print(f"CSV converted to Excel and saved as '{excel_file}'.")
else:
    print(f"CSV file '{csv_file}' does not exist.")
    exit()

# Step 2: Calculate proportions per cell starting from column 'O'
# Reload the Excel file with the added tab
df = pd.read_excel(excel_file, sheet_name='final_merged_output')

# Identify columns starting from column 'O'
start_column = 'Irrigated Temperate Corn'
start_index = df.columns.get_loc(start_column)

# Calculate proportions for each cell in columns starting from 'O'
proportions = df.iloc[:, start_index:].div(df.iloc[:, start_index:].sum(axis=1), axis=0)

# Add proportions to the DataFrame with appropriate headers
proportion_columns = [f"{col}" for col in df.columns[start_index:]]
proportions.columns = proportion_columns

# Step 3: Save to a new tab in the Excel file
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer:
    proportions.to_excel(writer, sheet_name='Proportions', index=False)
print(f"Proportions added as a new tab in '{excel_file}'.")


Spatial Aggregations

In [ ]:
# File paths
output_excel = r".\LUH2\MSE calculation\output.xlsx"
raw_classification_excel = r".\LUH2\MSE calculation\Raw Classification.xlsx"

# Step 1: Load the Proportions tab from output.xlsx
proportions_df = pd.read_excel(output_excel, sheet_name='Proportions')

# Step 2: Load the Raw Classification table
classification_df = pd.read_excel(raw_classification_excel)

# Step 3: Prepare the mapping
classification_df['Layer Name'] = classification_df['Layer Name'].str.strip()
classification_df['LUH2 Classification'] = classification_df['LUH2 Classification'].str.strip()

# Melt the Proportions DataFrame for easier aggregation
proportions_long = proportions_df.melt(ignore_index=False, var_name='Layer Name', value_name='Proportion')

# Merge with the classification mapping
merged_df = pd.merge(proportions_long.reset_index(), classification_df, on='Layer Name', how='inner')

# Step 4: Aggregate proportions per cell grouped by LUH2 Classification
aggregated_df = merged_df.groupby(['index', 'LUH2 Classification'])['Proportion'].sum().unstack(fill_value=0)

# Step 5: Handle columns with "_Non_Forested" and "Forested_"
# Identify and combine Non_Forested columns
non_forested_cols = [col for col in aggregated_df.columns if 'Non_Forested_' in col]
aggregated_df['Non_Forested'] = aggregated_df[non_forested_cols].sum(axis=1)
aggregated_df.drop(non_forested_cols, axis=1, inplace=True)

# Identify and combine Forested columns
forested_cols = [col for col in aggregated_df.columns if 'Forested_' in col]
aggregated_df['Forested'] = aggregated_df[forested_cols].sum(axis=1)
aggregated_df.drop(forested_cols, axis=1, inplace=True)

# Step 6: Save to the output Excel file
with pd.ExcelWriter(output_excel, engine='openpyxl', mode='a') as writer:
    aggregated_df.to_excel(writer, sheet_name='Aggregation')

print(f"Aggregation tab created successfully in '{output_excel}'.")

MSE Calculations

In [ ]:
# File path
output_excel = r".\LUH2\MSE calculation\output.xlsx"

# Step 1: Load the Aggregation and final_merged_output tabs
aggregation_df = pd.read_excel(output_excel, sheet_name='Aggregation')
final_merged_output_df = pd.read_excel(output_excel, sheet_name='final_merged_output')

# Step 2: Preprocess final_merged_output
# Combine all columns with "_Non_Forested" into a single column named "Non_Forested"
non_forested_columns = [col for col in final_merged_output_df.columns if "_Non_Forested" in col]
final_merged_output_df['Non_Forested'] = final_merged_output_df[non_forested_columns].sum(axis=1)
final_merged_output_df.drop(columns=non_forested_columns, inplace=True)

# Combine all columns with "Forested_" excluding those with "Non_Forested_" into a single column named "Forested"
forested_columns = [col for col in final_merged_output_df.columns if "Forested_" in col and "Non_Forested_" not in col]
final_merged_output_df['Forested'] = final_merged_output_df[forested_columns].sum(axis=1)
final_merged_output_df.drop(columns=forested_columns, inplace=True)

# Step 3: Ensure both DataFrames have matching columns
# Find common columns between Aggregation and processed final_merged_output
common_columns = list(set(aggregation_df.columns).intersection(final_merged_output_df.columns))

# Step 4: Perform the MSE calculation
# Subset the DataFrames to only include the common columns
aggregation_subset = aggregation_df[common_columns]
final_merged_subset = final_merged_output_df[common_columns]

# Apply the MSE formula: (Aggregation - (final_merged_output / 100))^2
mse_df = (aggregation_subset - (final_merged_subset / 100)) ** 2

# Step 4: Sum all columns per row, divide by 100, and add the result to a new column
mse_df['MSE_Sum_Per_Row'] = mse_df.sum(axis=1) / 100

# Step 5: Save the MSE results to a new tab in output.xlsx
with pd.ExcelWriter(output_excel, engine='openpyxl', mode='a') as writer:
    mse_df.to_excel(writer, sheet_name='MSE calculation', index=False)

print(f"MSE calculation tab created successfully in '{output_excel}'.")

MSE CSV

In [ ]:
# File path to the Excel file
output_excel = r".\LUH2\MSE calculation\output.xlsx"
output_csv = r".\LUH2\MSE calculation\pointid_mse.csv"

# Step 1: Load the relevant tabs from the Excel file
final_merged_output_df = pd.read_excel(output_excel, sheet_name='final_merged_output')
mse_calculation_df = pd.read_excel(output_excel, sheet_name='MSE calculation')

# Step 2: Extract the "point id" column and the "MSE_Sum_Per_Row" column
# Assuming "point id" exists in final_merged_output_df and "MSE_Sum_Per_Row" in mse_calculation_df
if 'pointid' in final_merged_output_df.columns:
    point_id_column = final_merged_output_df['pointid']
else:
    raise ValueError("'point id' column not found in final_merged_output tab.")

if 'MSE_Sum_Per_Row' in mse_calculation_df.columns:
    mse_sum_column = mse_calculation_df['MSE_Sum_Per_Row']
else:
    raise ValueError("'MSE_Sum_Per_Row' column not found in MSE calculation tab.")

# Step 3: Combine the columns into a new DataFrame
output_df = pd.DataFrame({
    'point id': point_id_column,
    'MSE_Sum_Per_Row': mse_sum_column
})

# Step 4: Save the new DataFrame to a CSV file
output_df.to_csv(output_csv, index=False)

output_csv